In [ ]:
!pip install gradio
!pip install pandas
!pip install matplotlib
!pip install ollama
!pip install pydantic

import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
import ollama
from pydantic import BaseModel

def validate_csv(file):
    try:
        df = pd.read_csv(file.name)
        return df, "CSV file uploaded and validated successfully!"
    except Exception as e:
        return None, f"Error: {str(e)}"

class QueryModel(BaseModel):
    question: str

def answer_question(df, question):
    try:
        # Prepare the query for the LLM
        query = f"Given the following dataset: {df.head().to_string()}, answer this question: {question}"
        
        # Use Ollama to generate the response
        response = ollama.generate(model="llama3.1-8b", prompt=query)
        
        return response['response']
    except Exception as e:
        return f"Error processing the question: {str(e)}"

def plot_graph(df, column_x, column_y):
    try:
        plt.figure(figsize=(8, 6))
        plt.plot(df[column_x], df[column_y], marker='o')
        plt.xlabel(column_x)
        plt.ylabel(column_y)
        plt.title(f"{column_y} vs {column_x}")
        plt.grid(True)
        return plt
    except Exception as e:
        return f"Error plotting the graph: {str(e)}"

def gradio_app(file, question, column_x, column_y):
    # Step 1: Validate and load the CSV file
    df, message = validate_csv(file)
    if df is None:
        return message
    
    # Step 2: Answer the user's question
    answer = answer_question(df, question)
    
    # Step 3: Plot the graph
    plot = plot_graph(df, column_x, column_y)
    
    # Return the results
    if isinstance(plot, plt.Figure):
        plt.close()
        return answer, plot
    else:
        return answer, plot

# Define Gradio inputs and outputs
inputs = [
    gr.File(label="Upload CSV File"),
    gr.Textbox(label="Ask a Question"),
    gr.Textbox(label="X-axis Column Name"),
    gr.Textbox(label="Y-axis Column Name")
]
outputs = [
    gr.Textbox(label="Answer"),
    gr.Plot(label="Graph")
]

# Launch the Gradio app
app = gr.Interface(fn=gradio_app, inputs=inputs, outputs=outputs, title="CSV Question Answering and Visualization")
app.launch() 